(d'après le tutoriel de [Michael P. Notter](https://miykael.github.io/blog/2022/advanced_eda/))

Comment maîtriser rapidement la plupart des ensembles de données tabulaires ?

Il n'est pas toujours facile de se faire une idée précise d'un nouvel ensemble de données, et cela prend du temps. Cependant, une bonne analyse des données exploratoire (ADE) peut être d'une grande aide pour comprendre votre jeu de données, avoir une idée de la façon dont les choses sont connectées et de ce qui doit être fait pour traiter correctement votre jeu de données.

Dans cet article, nous aborderons plusieurs routines utiles. Cependant, pour rester bref et compact, nous n'approfondirons pas toujours les choses ou n'expliquerons pas toutes les implications. Mais en réalité, passer suffisamment de temps sur une ADE appropriée pour comprendre pleinement votre ensemble de données est un élément clé de tout bon projet de science des données. En règle générale, vous passerez probablement 80 % de votre temps à la préparation et à l'exploration des données et seulement 20 % à la modélisation d'apprentissage automatique proprement dite.

Dans l'ensemble, l'analyse des données exploratoire est très itérative. À la fin de votre étude, vous découvrirez peut-être quelque chose qui vous obligera à tout refaire une fois de plus. C'est normal ! Mais je propose l'organisation suivante :

1. **Etude de la structure** : Exploration de la forme générale de l'ensemble de données, ainsi que des types de données de vos caractéristiques.
2. **Etude de la qualité** : Etude de la qualité générale de l'ensemble de données, en ce qui concerne les doublons, les valeurs manquantes et les entrées indésirables.
3. **Etude du contenu** : Une fois que la structure et la qualité de l'ensemble de données sont comprises, nous pouvons procéder à une exploration plus approfondie des valeurs des caractéristiques et examiner comment les différentes caractéristiques sont liées les unes aux autres.

Mais d'abord, nous devons trouver un jeu de données intéressant. Allons-y et chargeons le [jeu de données sur la sécurité routière](https://www.openml.org/d/42803) depuis [OpenML](https://www.openml.org/search?type=data).

In [ ]:
from sklearn.datasets import fetch_openml

# Download the dataset from openml
dataset = fetch_openml(data_id=42803, as_frame=True)

# Extract feature matrix X and show 5 random samples
df_X = dataset["frame"]
df_X.sample(5)

# Etude de la structure

Avant d'étudier le contenu de notre matrice de caractéristiques $X$, examinons d'abord la structure générale de l'ensemble de données. Par exemple, combien de colonnes et de lignes l'ensemble de données comporte-t-il ?

In [ ]:
# Show size of the dataset
df_X.shape

```
## (363243, 67)
```

Nous savons donc que cet ensemble de données comporte 363'243 échantillons et 67 caractéristiques. Et combien de types de données différents ces 67 caractéristiques contiennent-elles ?

In [ ]:
import pandas as pd

# Count how many times each data type is present in the dataset
pd.value_counts(df_X.dtypes)

```
## float64    61
## object      6
## dtype: int64
```

## Structure des caractéristiques non-numériques

Les types de données peuvent être numériques et non numériques. Tout d'abord, examinons de plus près les entrées **non numériques**.

In [ ]:
# Display non-numerical features
df_X.select_dtypes(exclude="number").head()

```
##      Accident_Index	Sex_of_Driver	Date	    Time	Local_Authority_(Highway)	LSOA_of_Accident_Location
## 0	201501BS70001	1.0	            12/01/2015	18:45	E09000020	                E01002825
## 1	201501BS70002	1.0	            12/01/2015	07:50	E09000020	                E01002820
## 2	201501BS70004	1.0	            12/01/2015	18:08	E09000020	                E01002833
## 3	201501BS70005	1.0	            13/01/2015	07:40	E09000020	                E01002874
## 4	201501BS70008	1.0	            09/01/2015	07:30	E09000020	                E01002814
```

Bien que `Sex_of_Driver` soit une caractéristique numérique, elle a été enregistrée comme une caractéristique non numérique. Ceci est parfois dû à une erreur dans l'enregistrement des données. Nous allons donc nous occuper de cela :

In [ ]:
# Changes data type of 'Sex_of_Driver'
df_X["Sex_of_Driver"] = df_X["Sex_of_Driver"].astype("float")

En utilisant la fonction `.describe()`, nous pouvons également étudier le nombre de valeurs uniques de chaque caractéristique non-numérique et la fréquence à laquelle la valeur la plus importante est présente.

In [ ]:
df_X.describe(exclude="number")

```
##          Accident_Index	Date	    Time	Local_Authority_(Highway)	LSOA_of_Accident_Location
## count	363243	        319866	    319822	319866	                    298758
## unique	140056	        365	        1439	204	                        25979
## top	    201543P296025	14/02/2015	17:30	E10000017	                E01028497
## freq	    1332	        2144	    2972	8457	                    1456
```

## Structure des caractéristiques numériques

Ensuite, examinons de plus près les caractéristiques numériques. Plus précisément, examinons combien de valeurs uniques possède chacune de ces caractéristiques. Ce processus nous donnera un aperçu du nombre de caractéristiques **binaires** (2 valeurs uniques), **ordinaires** (3 à ~10 valeurs uniques) et **continues** (plus de 10 valeurs uniques) dans l'ensemble de données.

In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df_X.select_dtypes(include="number").nunique().sort_values()

# Plot information with y-axis in log-scale
unique_values.plot.bar(logy=True, figsize=(15, 4), title="Unique values per feature")
plt.savefig("unique.png", bbox_inches='tight')

![unique](unique.png)

## Conclusion sur l'étude de la structure

À la fin de cette première enquête, nous devrions avoir une meilleure compréhension de la structure générale de notre ensemble de données. Le nombre d'échantillons et de caractéristiques, le type de données que chaque caractéristique possède, et combien d'entre elles sont binaires, ordinales, catégoriques ou continues. Pour obtenir ce genre d'informations, vous pouvez également utiliser `df_X.info()` ou `df_X.describe()`.

# Etude de la qualité

Avant de nous concentrer sur le contenu réel stocké dans ces caractéristiques, examinons d'abord la qualité générale de l'ensemble de données. L'objectif est d'avoir une vue globale de l'ensemble de données en ce qui concerne des éléments tels que les doublons, les valeurs manquantes et les entrées indésirables ou les erreurs d'enregistrement.

## Doublons

Les doublons sont des entrées qui représentent plusieurs fois le même point d'échantillonnage. Par exemple, si une mesure a été enregistrée deux fois par deux personnes différentes. La détection de ces doublons n'est pas toujours facile, car chaque ensemble de données peut avoir un identifiant unique (par exemple, un numéro d'index ou une heure d'enregistrement qui est unique à chaque nouvel échantillon) que vous pouvez vouloir ignorer dans un premier temps.

In [ ]:
# Check number of duplicates while ignoring the index feature
n_duplicates = df_X.drop(labels=["Accident_Index"], axis=1).duplicated().sum()
print(f"You seem to have {n_duplicates} duplicates in your database.")

```
## You seem to have 22 duplicates in your database.
```

Pour gérer ces doublons, vous pouvez tout simplement les supprimer avec `.drop_duplicates()`.

In [ ]:
#  Extract column names of all features, except 'Accident_Index'
columns_to_consider = df_X.drop(labels=["Accident_Index"], axis=1).columns

# Drop duplicates based on 'columns_to_consider'
df_X = df_X.drop_duplicates(subset=columns_to_consider)
df_X.shape

```
## (363221, 67)
```

## Valeurs manquantes

Les valeurs manquantes sont un autre problème de qualité qui mérite d'être étudié. Il est normal d'avoir quelques valeurs manquantes. Ce que nous voulons identifier à ce stade, ce sont les grands trous dans l'ensemble de données, c'est-à-dire les échantillons ou les caractéristiques avec un grand nombre de valeurs manquantes.

### Par échantillon

Pour regarder le nombre de valeurs manquantes par échantillon, nous avons plusieurs options. La plus simple est de simplement visualiser la sortie de `df_X.isna()`, avec quelque chose comme ceci :

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(df_X.isna(), aspect="auto", interpolation="nearest", cmap="gray")
plt.xlabel("Column Number")
plt.ylabel("Sample Number")
plt.savefig("isna.png", bbox_inches='tight')

![isna](isna.png)

Cette figure montre sur l'axe des y chacun des 360'000 échantillons individuels, et sur l'axe des x si l'une des 67 caractéristiques contient une valeur manquante. Bien que ce graphique soit déjà utile, une meilleure approche consiste à utiliser la bibliothèque [missingno](https://github.com/ResidentMario/missingno), pour obtenir un graphique comme celui-ci :

In [ ]:
import missingno as msno

msno.matrix(df_X, labels=True, sort="descending")
plt.savefig("missingno.png", bbox_inches='tight')

![missingno](missingno.png)

À partir de ces deux graphiques, nous pouvons voir que l'ensemble de données a un énorme vide, causé par certains échantillons où plus de 50% des valeurs de caractéristiques sont manquantes. Pour ces échantillons, remplir les valeurs manquantes avec des valeurs de remplacement n'est probablement pas une bonne idée.

Par conséquent, allons-y et éliminons les échantillons qui ont plus de 20% de valeurs manquantes. Le seuil est inspiré des informations de la colonne 'Exhaustivité des données' à droite de cette figure.

In [ ]:
df_X = df_X.dropna(thresh=df_X.shape[1] * 0.80, axis=0).reset_index(drop=True)
df_X.shape

```
## (319790, 67)
```

### Par caractéristiques

Regardons maintenant le nombre de valeurs manquantes par caractéristique. Pour cela, nous pouvons utiliser une astuce `pandas` pour identifier rapidement le ratio de valeurs manquantes **par caractéristique**.

In [ ]:
df_X.isna().mean().sort_values().plot(
    kind="bar", figsize=(15, 4),
    title="Percentage of missing values per feature",
    ylabel="Ratio of missing values per feature")
plt.savefig("miss.png", bbox_inches='tight')

![miss](miss.png)

Sur cette figure, nous pouvons voir que la plupart des caractéristiques ne contiennent pas de valeurs manquantes. Néanmoins, des caractéristiques comme `2nd_Road_Class`, `Junction_Control`, `Age_of_Vehicle` contiennent beaucoup de valeurs manquantes. Nous allons donc supprimer toutes les caractéristiques ayant plus de 15% de valeurs manquantes.

In [ ]:
df_X = df_X.dropna(thresh=df_X.shape[0] * 0.85, axis=1)
df_X.shape

```
## (319790, 60)
```

## Entrées indésirables ou erreurs d'enregistrement

Une autre source de problèmes de qualité dans un ensemble de données peut être due à des entrées non désirées ou à des erreurs d'enregistrement. Il est important de distinguer ces échantillons des simples valeurs aberrantes. Alors que les valeurs aberrantes sont des points de données qui sont inhabituels pour une distribution de caractéristiques donnée, les **entrées indésirables ou les erreurs d'enregistrement sont des échantillons qui ne devraient pas être là en premier lieu**.

Par exemple, un enregistrement de température de 45°C à Bordeaux peut être une valeur aberrante (comme dans "très inhabituel"), alors qu'un enregistrement à 90°C serait une erreur. De même, un enregistrement de la température au sommet du Pic du Midi pourrait être physiquement possible, mais ne devrait probablement pas être inclus dans un ensemble de données sur les villes de Gironde.

Bien entendu, la détection de telles erreurs et entrées indésirables et leur distinction avec les valeurs aberrantes ne sont pas toujours simples et dépendent fortement de l'ensemble de données. Une approche consiste à adopter une vue globale de l'ensemble de données et à voir si vous pouvez identifier des motifs très inhabituels.

### Caractéristiques numériques

Pour afficher cette vue globale du jeu de données, au moins pour les caractéristiques numériques, vous pouvez utiliser la fonction `.plot()` de `pandas` et la combiner avec les paramètres suivants :

- `lw=0` : `lw` représente la largeur du trait. `0` signifie que l'on ne veut pas montrer de lignes.
- `marker="."` : au lieu de lignes, on utiliser des `.` comme marqueurs pour chaque point de données.
- `subplots=True` : `subplots` indique à `pandas` de tracer chaque caractéristique dans un sous-graphe séparé.
- `layout=(-1, 4)` : indique à `pandas` le nombre de lignes et de colonnes à utiliser pour les sous-graphes. Le -1 signifie "autant que nécessaire", tandis que le 4 signifie utiliser 4 colonnes par ligne.
- `figsize=(15, 30), markersize=1` : pour s'assurer que la figure est assez grande, nous recommandons d'avoir une hauteur de figure d'environ le nombre de caractéristiques, et d'ajuster la `markersize` en conséquence.

Alors, à quoi ressemble ce graphique ?

In [ ]:
df_X.plot(lw=0, marker=".", subplots=True, layout=(-1, 4),
          figsize=(15, 30), markersize=1)
plt.savefig("num.png", bbox_inches='tight')

![num](num.png)

Ce type de graphique peut vous donner beaucoup d'idées pour le nettoyage des données. Il est généralement judicieux d'investir autant de temps que nécessaire jusqu'à ce que vous soyez satisfait du résultat de cette visualisation.

### Caractéristiques non-numériques

L'identification des **entrées indésirables** ou des **erreurs d'enregistrement** sur des caractéristiques non-numériques est un peu plus délicate. Nous pouvons jeter un coup d'œil général sur le nombre de valeurs uniques que chacune de ces caractéristiques non-numériques contient, et sur la fréquence à laquelle leur catégorie la plus fréquente est représentée.

In [ ]:
# Extract descriptive properties of non-numerical features
df_X.describe(exclude=["number", "datetime"])

```
##          Accident_Index	Date	    Time	Local_Authority_(Highway)	LSOA_of_Accident_Location
## count	319790	        319790	    319746	319790	                    298693
## unique	123645	        365	        1439	204	                        25977
## top	    201543P296025	14/02/2015	17:30	E10000017	                E01028497
## freq	    1332	        2144	    2969	8457	                    1456
```

Il existe de multiples façons de rationaliser l'étude de qualité pour chaque caractéristique non-numérique. Aucune d'entre elles n'est parfaite, et toutes nécessiteront une enquête complémentaire. Mais nous pourrions, par exemple, parcourir toutes les caractéristiques non-numériques et visualiser pour chacune d'elles le nombre d'occurrences par valeur unique.

In [ ]:
# Create figure object with 3 subplots
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(12, 8))

# Identify non-numerical features
df_non_numerical = df_X.select_dtypes(exclude=["number", "datetime"])

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_non_numerical.columns, axes.ravel()):

    # Selects one single feature and counts number of occurrences per unique value
    df_non_numerical[col].value_counts().plot(

        # Plots this information in a figure with log-scaled y-axis
        logy=True, title=col, lw=0, marker=".", ax=ax)

plt.tight_layout()
plt.savefig("non-num.png", bbox_inches='tight')

![non-num](non-num.png)

Nous pouvons voir que l'accident le plus fréquent (c'est-à-dire `Accident_Index`), avait plus de 100 personnes impliquées. En creusant un peu plus (c'est-à-dire en regardant les caractéristiques individuelles de cet accident), nous avons pu identifier que cet accident s'est produit le 24 février 2015 à 11:55 à Cardiff UK. Une recherche rapide sur Internet révèle que cette entrée correspond à un accident heureusement non mortel incluant un minibus rempli de retraités.

La décision sur ce qu'il convient de faire avec ces entrées plutôt singulières est une fois de plus laissée entre les mains subjectives de la personne qui analyse l'ensemble de données. Sans aucune justification valable, supprimons les 10 accidents les plus fréquents de cet ensemble de données.

In [ ]:
# Collect entry values of the 10 most frequent accidents
accident_ids = df_non_numerical["Accident_Index"].value_counts().head(10).index

# Removes accidents from the 'accident_ids' list
df_X = df_X[~df_X["Accident_Index"].isin(accident_ids)]
df_X.shape

```
## (317665, 60)
```

## Conclusion sur l'étude de la qualité

À la fin de cette étude, nous devrions avoir une meilleure compréhension de la qualité générale de notre ensemble de données. Nous avons examiné les doublons, les valeurs manquantes et les entrées indésirables ou les erreurs d'enregistrement. Il est important de souligner que nous n'avons pas encore discuté de la manière de traiter les valeurs manquantes ou aberrantes restantes dans l'ensemble de données. Il s'agit d'une tâche qui sera partiellement abordée dans la prochaine étude.

# Etude du contenu

Jusqu'à présent, nous n'avons examiné que la structure générale et la qualité de l'ensemble de données. Allons maintenant un peu plus loin et examinons le contenu réel. Dans un cadre idéal, une telle étude serait effectuée caractéristique par caractéristique. Mais cela devient très compliqué dès que l'on a plus de 20-30 caractéristiques.

Pour cette raison, nous allons explorer trois approches différentes qui peuvent vous donner un aperçu très rapide du contenu stocké dans chaque caractéristique et de la façon dont elles sont liées.

## Distribution des caractéristiques

L'examen de la distribution des valeurs de chaque caractéristique est un excellent moyen de mieux comprendre le contenu de vos données. En outre, cela peut fournir de nombreuses informations utiles en ce qui concerne le nettoyage des données et la transformation des caractéristiques. La façon la plus rapide de le faire pour les caractéristiques numériques est d'utiliser des histogrammes. Heureusement, `pandas` est livré avec une fonction histogramme intégrée qui permet de tracer plusieurs caractéristiques à la fois.

In [ ]:
# Plots the histogram for each numerical feature in a separate subplot
df_X.hist(bins=25, figsize=(15, 25), layout=(-1, 5), edgecolor="black")
plt.tight_layout()
plt.savefig("hist.png", bbox_inches='tight')

![hist](hist.png)

**Entrée la plus fréquente** : Certaines caractéristiques, telles que `Towing_and_Articulation` ou `Was_Vehicle_Left_Hand_Drive?` contiennent principalement des entrées d'une seule catégorie. En utilisant la fonction `.mode()`, nous pourrions par exemple extraire le ratio de l'entrée la plus fréquente pour chaque caractéristique et visualiser cette information.

In [ ]:
# Collects for each feature the most frequent entry
most_frequent_entry = df_X.mode()

# Checks for each entry if it contains the most frequent entry
df_freq = df_X.eq(most_frequent_entry.values, axis=1)

# Computes the mean of the 'is_most_frequent' occurrence
df_freq = df_freq.mean().sort_values(ascending=False)

# Show the 5 top features with the highest ratio of singular value content
display(df_freq.head())

# Visualize the 'df_freq' table
df_freq.plot.bar(figsize=(15, 4))
plt.savefig("freq.png", bbox_inches='tight')

![freq](freq.png)

**Distributions de valeurs asymétriques** : Certains types de caractéristiques numériques peuvent également présenter des distributions fortement non gaussiennes. Dans ce cas, vous pouvez réfléchir à la façon dont vous pouvez transformer ces valeurs pour qu'elles soient distribuées plus normalement.

## Motifs des caractéristiques

L'étape suivante est l'étude des motifs spécifiques aux caractéristiques. L'objectif de cette partie est double :

1. Pouvons-nous identifier des motifs particuliers au sein d'une caractéristique qui nous aideront à décider si certaines entrées doivent être abandonnées ou modifiées ?
2. Pouvons-nous identifier des relations particulières entre les caractéristiques qui nous aideront à mieux comprendre notre ensemble de données ?

Mais avant de nous plonger dans ces deux questions, examinons de plus près quelques caractéristiques "sélectionnées au hasard".

In [ ]:
df_X[["Location_Northing_OSGR", "1st_Road_Number",
      "Journey_Purpose_of_Driver", "Pedestrian_Crossing-Physical_Facilities"]].plot(
    lw=0, marker=".", subplots=True, layout=(-1, 2), markersize=0.1, figsize=(15, 6))
plt.savefig("motifs.png", bbox_inches='tight')

![motifs](motifs.png)

Dans la rangée du haut, nous pouvons voir des caractéristiques avec des valeurs continues, tandis que dans la rangée du bas, nous avons des caractéristiques avec des valeurs discrètes.

Bien qu'il existe de nombreuses façons d'explorer nos caractéristiques à la recherche de motifs particuliers, simplifions en décidant que nous traitons les caractéristiques avec moins de 25 caractéristiques uniques comme des caractéristiques **discrètes** ou **ordinaires**, et les autres caractéristiques comme des caractéristiques **continues**.

In [ ]:
# Creates mask to identify numerical features with more or less than 25 unique features
cols_continuous = df_X.select_dtypes(include="number").nunique() >= 25

### Caractéristiques continues

Maintenant que nous avons un moyen de sélectionner les caractéristiques continues, utilisons le `pairplot` de `seaborn` pour visualiser les relations entre ces caractéristiques. Il est **important** de noter que la routine `pairplot` de `seaborn` peut prendre beaucoup de temps pour créer tous les sous-graphes. Par conséquent, nous recommandons de ne pas l'utiliser pour plus d'une dizaine caractéristiques à la fois.

In [ ]:
# Create a new dataframe which only contains the continuous features
df_continuous = df_X[cols_continuous[cols_continuous].index]
df_continuous.shape

```
## (317665, 11)
```

Étant donné que dans notre cas, nous n'avons que 11 caractéristiques, nous pouvons continuer avec le diagramme par paire. Sinon, l'utilisation de quelque chose comme `df_continuous.iloc[ :, :5]` pourrait aider à réduire le nombre de caractéristiques à tracer.

In [ ]:
import seaborn as sns

sns.pairplot(df_continuous, height=1.5, plot_kws={"s": 2, "alpha": 0.2})
plt.savefig("pairplot.png", bbox_inches='tight')

![pairplot](pairplot.png)

Il semble y avoir une relation étrange entre quelques caractéristiques dans le coin supérieur gauche. `Location_Easting_OSGR` et `Longitude`, ainsi que `Location_Easting_OSGR` et `Latitude` semblent avoir une relation linéaire très forte.

In [ ]:
sns.pairplot(
    df_X, plot_kws={"s": 3, "alpha": 0.2}, hue="Police_Force", palette="Spectral",
    x_vars=["Location_Easting_OSGR", "Location_Northing_OSGR", "Longitude"],
    y_vars="Latitude")
plt.savefig("pairplot2.png", bbox_inches='tight')

![pairplot2](pairplot2.png)

Sachant que ces caractéristiques contiennent des informations géographiques, une étude plus approfondie concernant la géolocalisation pourrait être fructueuse. Cependant, pour l'instant, nous poursuivons l'exploration des caractéristiques discrètes et ordinales.

### Caractéristiques discrètes ou ordinaires

Trouver des modifs dans les caractéristiques discrètes ou ordinales est un peu plus délicat. Mais ici aussi, quelques astuces rapides avec `pandas` et `seaborn` peuvent nous aider à obtenir une vue d'ensemble de notre jeu de données. Tout d'abord, sélectionnons les colonnes que nous voulons étudier.

In [ ]:
# Create a new dataframe which doesn't contain the numerical continuous features
df_discrete = df_X[cols_continuous[~cols_continuous].index]
df_discrete.shape

```
## (317665, 44)
```

Comme souvent, il existe de multiples façons d'étudier toutes ces fonctionnalités. Essayons `stripplot()` de `seaborn` avec une boucle 'for' pratique `zip()` pour les sous-graphes.

**Remarque**, pour répartir les valeurs dans la direction de l'axe des y, nous devons choisir une caractéristique particulière (en espérant qu'elle soit instructive). Bien que la "bonne" caractéristique puisse aider à identifier des motifs intéressants, toute caractéristique continue devrait généralement faire l'affaire. Le principal intérêt de ce type de graphique est de voir combien d'échantillons chaque valeur discrète contient.

In [ ]:
import numpy as np

# Establish number of columns and rows needed to plot all features
n_cols = 5
n_elements = len(df_discrete.columns)
n_rows = np.ceil(n_elements / n_cols).astype("int")

# Specify y_value to spread data (ideally a continuous feature)
y_value = df_X["Age_of_Driver"]

# Create figure object with as many rows and columns as needed
fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(15, n_rows * 2.5))

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_discrete.columns, axes.ravel()):
    sns.stripplot(data=df_X, x=col, y=y_value, ax=ax, palette="tab10", size=1, alpha=0.5)
plt.tight_layout()
plt.savefig("stripplot.png", bbox_inches='tight')

![stripplot](stripplot.png)

Il y a trop de choses à commenter ici, alors concentrons-nous sur quelques-unes. En particulier, concentrons-nous sur 6 caractéristiques où les valeurs apparaissent dans un motif particulier ou où certaines catégories semblent être beaucoup moins fréquentes que d'autres. Et pour secouer un peu les choses, utilisons maintenant la caractéristique `Latitude` pour étirer les valeurs sur l'axe des ordonnées.

In [ ]:
# Specify features of interest
selected_features = ["Vehicle_Reference_df_res", "Towing_and_Articulation",
                     "Skidding_and_Overturning", "Bus_or_Coach_Passenger",
                     "Pedestrian_Road_Maintenance_Worker", "Age_Band_of_Driver"]

# Create a figure with 3 x 2 subplots
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(16, 8))

# Loop through these features and plot entries from each feature against `Latitude`
for col, ax in zip(selected_features, axes.ravel()):
    sns.stripplot(data=df_X, x=col, y=df_X["Latitude"], ax=ax,
                  palette="tab10", size=2, alpha=0.5)
plt.tight_layout()
plt.savefig("stripplot2.png", bbox_inches='tight')

![stripplot2](stripplot2.png)

Ce type de graphique est déjà très instructif, mais il masque les régions où il y a beaucoup de points de données à la fois. Par exemple, il semble y avoir une forte densité de points dans certains des graphiques à la 52e latitude. Regardons donc de plus près avec un graphe approprié, tel que `violineplot` (ou `boxenplot` ou `boxplot`). Et pour aller plus loin, séparons également chaque visualisation par `Urban_ou_Rural_Area`.

In [ ]:
# Create a figure with 3 x 2 subplots
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(16, 8))

# Loop through these features and plot entries from each feature against `Latitude`
for col, ax in zip(selected_features, axes.ravel()):
    sns.violinplot(data=df_X, x=col, y=df_X["Latitude"], palette="Set2",
                   split=True, hue="Urban_or_Rural_Area", ax=ax)
plt.tight_layout()
plt.savefig("violinplot.png", bbox_inches='tight')

![violinplot](violinplot.png)

Intéressant ! Nous pouvons voir que certaines valeurs sur les caractéristiques sont plus fréquentes dans les zones urbaines, que dans les zones rurales (et vice versa). De plus, comme on le soupçonnait, il semble y avoir un pic de densité élevé à la latitude 51,5. Cela est très probablement dû à une région plus densément peuplée autour de Londres (à 51,5074°).

## Relations entre caractéristiques

Enfin, examinons les relations entre les caractéristiques. Plus précisément, comment elles sont corrélées. La façon la plus rapide de le faire est d'utiliser la fonction `.corr()` de `pandas`. Calculons donc la matrice de corrélation entre caractéristiques pour toutes les caractéristiques numériques.

In [ ]:
# Computes feature correlation
df_corr = df_X.corr(method="pearson")

**Remarque** : Selon le jeu de données et le type de caractéristiques (par exemple, des caractéristiques ordinales ou continues), vous pouvez utiliser la méthode `spearman` au lieu de la méthode `pearson` pour calculer la corrélation. Alors que la corrélation **Pearson** évalue la relation linéaire entre deux variables continues, la corrélation de **Spearman** est utilisée lorsque deux variables statistiques semblent corrélées sans que la relation entre les deux variables soit de type affine. Elle consiste à trouver un coefficient de corrélation, non pas entre les valeurs prises par les deux variables mais entre les rangs de ces valeurs (voir [wikipedia](https://fr.wikipedia.org/wiki/Corr%C3%A9lation_de_Spearman)).

Pour aider à l'interprétation de cette matrice de corrélation, utilisons le `.heatmap()` de `seaborn` pour la visualiser.

In [ ]:
# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8})
plt.savefig("heatmap.png", bbox_inches='tight')

![heatmap](heatmap.png)

Cela semble déjà très intéressant. Nous pouvons voir quelques corrélations très fortes entre certaines des caractéristiques. Maintenant, si vous êtes intéressé à ordonner toutes ces différentes corrélations, vous pouvez faire quelque chose comme ça :

In [ ]:
#  Creates a mask to remove the diagonal and the upper triangle.
lower_triangle_mask = np.tril(np.ones(df_corr.shape), k=-1).astype("bool")

#  Stack all correlations, after applying the mask
df_corr_stacked = df_corr.where(lower_triangle_mask).stack().sort_values()

#  Showing the lowest and highest correlations in the correlation matrix
display(df_corr_stacked)

Comme vous pouvez le constater, l'étude des corrélations entre les caractéristiques peut être très instructive. Mais le fait de tout regarder en même temps peut parfois être plus déroutant qu'utile. Ainsi, se concentrer sur une seule caractéristique avec quelque chose comme `df_X.corrwith(df_X["Speed_limit"])` pourrait être une meilleure approche.

De plus, les corrélations peuvent être trompeuses si une caractéristique contient encore beaucoup de valeurs manquantes ou de valeurs aberrantes extrêmes. Par conséquent, il est toujours important de s'assurer que votre matrice de caractéristiques est correctement préparée avant d'étudier ces corrélations.

## Conclusion sur l'étude du contenu

À la fin de cette troisième étude, nous devrions avoir une meilleure compréhension du contenu de notre jeu de données. Nous avons examiné la distribution des valeurs, les motifs et les corrélations de ces caractéristiques. Cependant, il ne s'agit certainement pas de toutes les options possibles d'investigation du contenu et de nettoyage des données que vous pouvez effectuer.